In [2]:

import json
import os

import gspread
import pandas as pd

def get_worksheet(ws_name):
    credentials_dict_str = os.environ["GOOGLE_CREDENTIALS"]
    credentials_dict = json.loads(credentials_dict_str)
    g_credentials = gspread.service_account_from_dict(credentials_dict)
    g_sheet = g_credentials.open("Korea - Users stats")
    worksheet = g_sheet.worksheet(ws_name)

    return worksheet


def update_worksheet(dataframe, worksheet):
    dataframe = dataframe.fillna("")
    df_list = [dataframe.columns.values.tolist()]
    df_list += dataframe.values.tolist()
    worksheet.update(df_list, value_input_option="USER_ENTERED")



In [3]:

vocab_g_ws = get_worksheet("Freezpmark")
df = pd.DataFrame(vocab_g_ws.get_all_records())

from datetime import datetime
from statistics import fmean

def scoring(df):
    d = {}
    d["Score"] = df["Session number"].sum()
    return pd.Series(d, index=["Score"])

# df["Date"] - pd.to_datetime("now")
# df.groupby("Word").apply(scoring)
df.sort_values(["Word", "Date"], ascending=[True, False], inplace=True)



In [4]:
# Getting the current date and time
dt = datetime.now()
dt2 = "2023-01-13 07:49:19"
datetime_object = datetime.strptime(dt2, "%Y-%m-%d %H:%M:%S")
ts = (dt - datetime_object).days
ts

8

In [18]:
distr = (1, 0.8, 0.64, 0.512, 0.4096, 0.32768, 0.262144, 0.209715, 0.167772, 0.1342176)
scores = {i: j for i, j in enumerate(distr)}

datetime_now = datetime.now()

knowledge = []
word_scores = {}
new_word = ""
new_word_counter = 1
new_word_score = 0
for row in df.itertuples():
    if new_word_counter >= 10:
        continue  # needs opt -> use pd
    if new_word == row.Word:
        new_word_counter += 1
    else:
        if new_word:
            # empty_fill = sum(4*x for x in distr[new_word_counter+1::])
            empty_fill = (fmean(knowledge) * (10-new_word_counter))  # mean
            word_scores[new_word] = new_word_score + empty_fill
            knowledge = []
            new_word_score = 0
            new_word_counter = 1
        new_word = row.Word

    if row.Knowledge == "✅":
        knowledge_multiplier = 1
    elif row.Knowledge == "⏭️":
        knowledge_multiplier = 2
    elif row.Knowledge == "❌":
        knowledge_multiplier = 4
    # new_word_score.append(f"{knowledge_multiplier}*{scores[new_word_counter]}")
    
    new_word_score += knowledge_multiplier * scores[new_word_counter]
    knowledge.append(knowledge_multiplier)

    row_datetime = datetime.strptime(row.Date, "%Y-%m-%d %H:%M:%S")
    now_datetime = datetime_now
    days_diff = (now_datetime - row_datetime).days
    new_word_score += days_diff * 0.01

    if row.Word == "열쇠" or row.Word == "뭐1":
        print(f"{row.Word}: {knowledge_multiplier}*{scores[new_word_counter]};; {new_word_score}")


sorted_word_score = dict(sorted(word_scores.items(), key=lambda item: item[1]))
sorted_word_score


뭐1: 1*0.8;; 0.88
뭐1: 1*0.64;; 1.61
열쇠: 2*0.8;; 1.6800000000000002
열쇠: 1*0.64;; 2.41


{'좋아하다': 9.182,
 '가수': 9.192,
 '멕시코': 9.192,
 '씨1': 9.192,
 '공부': 9.6,
 '공부하다': 9.6,
 '공책': 9.6,
 '기다리다': 9.6,
 '나라': 9.6,
 '남자': 9.6,
 '네1': 9.6,
 '누구': 9.6,
 '디자이너': 9.6,
 '먹다1': 9.6,
 '미국': 9.6,
 '미용사': 9.6,
 '배우': 9.6,
 '베트남': 9.6,
 '비싸다': 9.6,
 '사람': 9.6,
 '시계': 9.6,
 '아니다': 9.6,
 '어느': 9.6,
 '여자': 9.6,
 '연필': 9.6,
 '영국': 9.6,
 '영어': 9.6,
 '옷': 9.6,
 '요리사': 9.6,
 '유학생': 9.6,
 '은행원': 9.6,
 '이것': 9.6,
 '이름': 9.6,
 '일본': 9.6,
 '저1': 9.6,
 '저것': 9.6,
 '좋다': 9.6,
 '주다1': 9.6,
 '주부': 9.6,
 '중국': 9.6,
 '지우개': 9.6,
 '직업': 9.6,
 '책': 9.6,
 '처음': 9.6,
 '카메라': 9.6,
 '커피': 9.6,
 '프랑스': 9.6,
 '필리핀': 9.6,
 '학생': 9.6,
 '한국': 9.6,
 '한국어': 9.6,
 '회사원': 9.6,
 '뭐1': 9.61,
 '가다1': 9.88,
 '가르치다': 9.88,
 '가방': 9.88,
 '거울': 9.88,
 '것': 9.88,
 '교실': 9.88,
 '그것': 9.88,
 '깨끗하다': 9.88,
 '꽃': 9.88,
 '나쁘다': 9.88,
 '날씨': 9.88,
 '닫다': 9.88,
 '도서관': 9.88,
 '마시다': 9.88,
 '많다': 9.88,
 '맛없다': 9.88,
 '맛있다': 9.88,
 '머리': 9.88,
 '멋있다': 9.88,
 '모자': 9.88,
 '무엇': 9.88,
 '문': 9.88,
 '물': 9.88,
 '바나나': 9.88,
 '바쁘다': 9.88,